# Putting it all together: Analyzing a Toy Dataset

In this example, we're working with an artificial dataset from a production process, where a small fraction of the produced products are faulty. The task is to predict from the conditions under which a product is to be produced, whether the product will be ok or scrap.

In [ ]:
# first load some libraries that are needed later
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
# machine learning stuff
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.inspection import PartialDependenceDisplay, permutation_importance
from sklearn import tree
# interactive plotting (parallel coordinate plot)
import plotly.express as px
# suppress unnecessary warnings
# import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)

## Loading the data

The data is available as a `.csv` file, which stands for "comma-separated values", which is just a text file with one data point per row. You can export this kind of format from Excel (thereby making the data easier to share) and then read it in with the `pandas` library.

The toy dataset consists of production data for 3 different types of products. The variables in the dataset are:
- `height`, `width`, `depth`: dimensions of the product
- `product`: categorical variable with values `1`, `5`, or `17` depending on the type of product that was produced
- `faulty`: binary variable that indicates if the produced product is faulty (`1`) or ok (`0`)

In [ ]:
# we are given the dataset toydata1.csv
# load the csv file into a dataframe with pandas
df = pd.read_csv("../data/toydata1.csv")
# look at the raw data (first 5 rows)
df.head()

In [ ]:
# more concise overview (e.g. how many values per column, mean of the values in each column, etc)
df.describe()

## Exploratory Analysis

To get a better feeling for what we're dealing with here, we examine the different variables in more detail.

- Do we have an equal amount of samples for each of the three product types or is one of the subgroups underrepresented?
- In what ranges are the features and are there differences amongst the three products?
- Are there correlations between the variables?
- Can we already identify some variables that tell us that a product is faulty?

In [ ]:
# plot histograms for the different variables
df.hist(bins=50, layout=(1, 5), figsize=(15, 2));

These histograms show the distribution of the values for each variable, i.e., on the x-axis you see the range of values and on the y-axis how many samples have a value in the respective interval.

**Take a second to examine these histograms - what do they already tell you?**
- Do we have to worry about underrepresented subgroups due to the different product types?
- Where might the 3 peaks in the distribution of the depth variable come from?
- What do you notice about the height and width variables?

In [ ]:
# verify counts for the categorical variable
df["product"].value_counts()

In [ ]:
# see if the variation in the depth variable is related to the different product types
plt.figure()
colors = ["r", "b", "g"]
# plot one histogram per product type using different colors
for i, prod in enumerate(sorted(df["product"].unique())):
    plt.hist(df["depth"][df["product"] == prod], bins=20, color=colors[i], alpha=0.7, label=f"product {prod}")
plt.legend()
plt.xlabel("depth");

In [ ]:
# look at the correlation matrix to see the correlations between all variables
# for more info on what these numbers mean see here: https://en.wikipedia.org/wiki/Correlation_and_dependence
corr_mat = df.corr()
# we add color to the table with .style
corr_mat.style.background_gradient(cmap='coolwarm', axis=None).format(precision=2)

We've already seen that the depth variable and product variable are connected, which explains their high correlation. The height and width variables also show a fairly high correlation of 0.72 and we had already seen that they also have very similar looking histograms, so lets investigate this further.

In [ ]:
# examine the correlation between height and width in more detail with a scatter plot
plt.figure(figsize=(5.5, 5))
plt.scatter(df["height"], df["width"], alpha=0.3)
plt.xlabel("height")
plt.ylabel("width")
plt.title(f"Correlation: {pearsonr(df['height'], df['width'])[0]:.3f}");  # just compute the same correlation again

**Questions:**
- If all that someone had told you was that two variables have a linear correlation of 0.7, is this the scatter plot that you would have imagined for the two variables? (You might also want to look at the Wikipedia article again for some other example plots)
- Why is the correlation coefficient for these two variables so large?
- What would you expect the correlation coefficient to be if you only consider the large blob in the middle (i.e., ignore the points at (0, 0))?

In reality, it often happens that two variables seem to be perfectly correlated (i.e., they have a correlation coefficient of (almost) 1), but when you look closer, then this is just due to the fact that, for example, two sensors are off at the same time, but for the part where they're on, they actually aren't giving redundant values. Therefore be careful before throwing away "rendundant" variables and always verify the correlation with a scatter plot!

In [ ]:
# now check if these variables already give a hint on how to identify the faulty products
# (they both also had a fairly high negative correlation with the faulty variable)
plt.figure()
plt.scatter(df["height"], df["width"], c=df["faulty"], alpha=0.3)  # color the points based on the faulty variable
plt.xlabel("height")
plt.ylabel("width")
plt.colorbar()
# and check what the correlation coefficient is without the (0, 0) points
plt.title(f"Correlation: {pearsonr(df['height'][df['height'] > 0], df['width'][df['width'] > 0])[0]:.3f}");

Clearly, not all faulty products are equal: some are within the "regular" data (i.e., the purple points), while some are outliers at (0, 0). 

The department that gave us the data tells us that the points where height=width=0 are products where something went wrong during production and the process was aborted. However, instead of marking the respective values as `NaN`, this was recorded by setting some of the variables to "impossible" values. Real data is just messy like that.

In [ ]:
# make an interactive parallel coordinate plot 
# (make sure you're using a modern browser for this, i.e., not the Internet Explorer!)
# (works with pandas as well, but doesn't look that great: pd.plotting.parallel_coordinates(df, "faulty"))
fig = px.parallel_coordinates(df, color="faulty")
fig
# each line corresponds to one sample, where the indivdual values for each variable are marked
# at the respective axis and then these values are connected by a line
# -> you can select parts of the samples by clicking and draging the mouse over one of the axis (when you see a cross)
# e.g., try to select only those samples that do not have a height and width of 0
# (a click on the selection removes it again, you can also drag the axis to change their order)
# do you notice any patterns?

## Supervised Learning

Now that we've become more familiar with the dataset, it's time to tackle the real task, i.e., to try to predict whether a product will be faulty. This is a classification problem (each product either belongs to the class "faulty" or the class "ok", there is no in between).

In [ ]:
# "product" is a categorical variable; for it to be handled correctly,
# we have to transform it into a one-hot encoded vector
e = OneHotEncoder(sparse_output=False, categories='auto')
ohe = e.fit_transform(df[["product"]])
df = df.join(pd.DataFrame(ohe, columns=[f"product_{i}" for i in e.categories_[0]], index=df.index))
df.head()  # notice the additional columns with zeros and a one

In [ ]:
# from the dataframe we now extract our features ...
feature_cols = ["product_1", "product_5", "product_17", "height", "width", "depth"]
X = df[feature_cols]
# ... and the vector with labels
y = df["faulty"]
# to evaluate our prediction model, we need to split off a test dataset
# later we will use the train_test_split function from sklearn to do this, 
# but this just goes to show that there is no magic behind it
np.random.seed(10)
idx = np.random.permutation(len(df))  # shuffled range of values from 0 to len(df)
train_idx = idx[:2000]  # 2/3 of the samples are in the training set
test_idx = idx[2000:]
X_train = X.loc[train_idx]  # pick out the rows from X corresponding to these indices
X_test = X.loc[test_idx]
y_train = y.loc[train_idx]
y_test = y.loc[test_idx]

In [ ]:
# see how imbalanced the label distribution in the training and test sets is
print(f"Fraction of ok items in training set: {1-np.mean(y_train):.3f}")
print(f"Fraction of ok items in test set: {1-np.mean(y_test):.3f}")
# and check the (balanced) accuracy for a stupid baseline model that always predicts zeros
# (notice how the value for the accuray is the same as the fraction of ok items above)
print("----- Stupid baseline (always predict 'ok'): -----")
print(f"Accuracy on training data: {accuracy_score(y_train, np.zeros_like(y_train)):.3f}")
print(f"Accuracy on test data: {accuracy_score(y_test, np.zeros_like(y_test)):.3f}")
print(f"Balanced accuracy on training data: {balanced_accuracy_score(y_train, np.zeros_like(y_train)):.3f}")
print(f"Balanced accuracy on test data: {balanced_accuracy_score(y_test, np.zeros_like(y_test)):.3f}")
# since we have a very unbalanced class distribution in this dataset, the balanced accuracy
# is the evaluation metric that we actually care about

In [ ]:
# let's try a (shallow) decision tree!
clf = tree.DecisionTreeClassifier(max_depth=2, random_state=1)
clf = clf.fit(X_train, y_train)
# same evaluation as for the stupid baseline above
print(f"Accuracy on training data: {clf.score(X_train, y_train):.3f}")
print(f"Accuracy on test data: {clf.score(X_test, y_test):.3f}")
print(f"Balanced accuracy on training data: {balanced_accuracy_score(y_train, clf.predict(X_train)):.3f}")
print(f"Balanced accuracy on test data: {balanced_accuracy_score(y_test, clf.predict(X_test)):.3f}")

**Questions:** 
- Have a look at the values for (balanced) accuracy and compare them to the scores obtained with the stupid baseline: Do you think we're on the right track, i.e., does this seem like a useful model?

In [ ]:
# now plot the tree
tree.plot_tree(clf, feature_names=feature_cols, filled=True, class_names=np.array(clf.classes_, dtype=str), proportion=True);

The decision tree has its root at the top (where you start) and the leaves (i.e., those nodes that don't branch off anymore) at the bottom (where you stop and make the final prediction). Each node in the tree shows in the first line the variable based on which the next split is made incl. the threshold value (except for leaf nodes), then the current Gini impurity (i.e., how homogeneous the labels of all the samples that ended up in this node are; this is what the decision tree internally optimizes, i.e., notice how the value gets smaller on at least one side after a split), then the fraction of samples that ended up in this node, and the distribution of samples into the different classes, as well as the class that would be predicted for a sample at this point.

**Questions:** 
- Have a look at the tree and the decisions that are made in it: What has the decision tree actually learned, i.e., which samples does it classify as faulty and which as ok? Does this model help us on our quest to identify production conditions that result in faulty products?

In [ ]:
# let's do what we probably should have done in the beginning and 
# remove the outliers (i.e., keep only samples with a height > 0)
df_new = df[df["height"] > 0.]
# create a train/test split again, this time using the sklearn function
X_train, X_test, y_train, y_test = train_test_split(df_new[feature_cols], 
                                                    df_new["faulty"], 
                                                    test_size=0.33, random_state=15)
# see how imbalanced the label distribution in the training and test sets is
print(f"Fraction of ok items in training set: {1-np.mean(y_train):.3f}")
print(f"Fraction of ok items in test set: {1-np.mean(y_test):.3f}")
# and what the stupid baselien is now (since we've removed only 'faulty' points, 
# the class distributions are even more unbalanced and the accuracy even higher)
print("----- Stupid baseline (always predict 'ok'): -----")
print(f"Accuracy on training data: {accuracy_score(y_train, np.zeros_like(y_train)):.3f}")
print(f"Accuracy on test data: {accuracy_score(y_test, np.zeros_like(y_test)):.3f}")
print(f"Balanced accuracy on training data: {balanced_accuracy_score(y_train, np.zeros_like(y_train)):.3f}")
print(f"Balanced accuracy on test data: {balanced_accuracy_score(y_test, np.zeros_like(y_test)):.3f}")

In [ ]:
# decision tree on data without outliers
clf = tree.DecisionTreeClassifier(max_depth=3, random_state=1)
clf = clf.fit(X_train, y_train)
print(f"Accuracy on training data: {clf.score(X_train, y_train):.3f}")
print(f"Accuracy on test data: {clf.score(X_test, y_test):.3f}")
print(f"Balanced accuracy on training data: {balanced_accuracy_score(y_train, clf.predict(X_train)):.3f}")
print(f"Balanced accuracy on test data: {balanced_accuracy_score(y_test, clf.predict(X_test)):.3f}")

In [ ]:
# plot the tree again
plt.figure(figsize=(15, 10))
tree.plot_tree(clf, feature_names=feature_cols, filled=True, class_names=np.array(clf.classes_, dtype=str), proportion=True);
# notice how in the leaf nodes where the tree predicts "faulty", there are only very few data points

**Questions:** 
- What do you think of the model now?

In [ ]:
# maybe we just need to give the tree the freedom to make more splits? (i.e., increase its depth)
clf = tree.DecisionTreeClassifier(max_depth=100, random_state=1)
clf = clf.fit(X_train, y_train)
print(f"Accuracy on training data: {clf.score(X_train, y_train):.3f}")
print(f"Accuracy on test data: {clf.score(X_test, y_test):.3f}")
print(f"Balanced accuracy on training data: {balanced_accuracy_score(y_train, clf.predict(X_train)):.3f}")
print(f"Balanced accuracy on test data: {balanced_accuracy_score(y_test, clf.predict(X_test)):.3f}")

**Questions:** 
- Is this a better model? If anything, is the model over- or underfitting?

In [ ]:
# to interpret the model when the tree is too large (or you're using a random forest),
# check the feature importances instead of plotting the tree
dict(zip(feature_cols, clf.feature_importances_))

In [ ]:
# similarly, we can also check the permutation feature importance
# to see which features are most important for the model (works for any model)
result = permutation_importance(clf, X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1)
sorted_idx = result.importances_mean.argsort()
plt.figure(figsize=(10, 6))
plt.boxplot(result.importances[sorted_idx].T, vert=False, labels=np.array(feature_cols)[sorted_idx])
plt.title("Permutation Importance");

### Garbage in, garbage out...

Clearly, we're missing some important information, as we are unable to identify the non-outlier faulty products. I.e., we need more data (not necessarily more samples, but certainly more features).

So we go back to the person that gave us the data and ask if they have an idea what else might be causing the products to break and if there are additional sensor measurements available that we could look at. They give us a new dataset `toydata2.csv`, which additionally contains the variable `temp`, which indicates the temperature at which a product was produced.

In [ ]:
# load this new data
df = pd.read_csv("../data/toydata2.csv")
df.head()  # same as before just an additional column

In [ ]:
# look at the variables again -> just like depth, temp has 3 peaks in the distribution
df.hist(bins=50, layout=(1,6), figsize=(15,2));

In [ ]:
# see if the variation in temp are indeed related to the different products
plt.figure()
colors = ["r", "b", "g"]
for i, prod in enumerate(sorted(df["product"].unique())):
    plt.hist(df["temp"][df["product"] == prod], bins=20, color=colors[i], alpha=0.7, label=f"product {prod}")
plt.legend()
plt.xlabel("temp");

In [ ]:
# make another interactive parallel coordinates plot
columns = ["height", "width", "depth", "product", "temp", "faulty"]
fig = px.parallel_coordinates(df[columns], color="temp")
fig

By clicking and dragging on the different axis, select the data such that you remove the outliers (i.e., keep only samples with height/width > 0) and then select the faulty products (i.e., with faulty = 1).

**Questions:** 
- Do you notice any patterns? How would you explain to the stakeholders why some of their products are faulty?

(In this case, we can derive the relevant insights already from the plot. However, in real problems, the solution is usually not this obvious, so lets try to see how we could also solve this with ML.)

### Supervised Learning 2.0

In [ ]:
# transform the categorical column again - this time using pandas directly
df = pd.concat([df, pd.get_dummies(df["product"], prefix="product")], axis=1)
df.head()

In [ ]:
# remove outliers again
df_new = df[df["height"] > 0.]
# let's try with temp as an additional feature
feature_cols = ["product_1", "product_5", "product_17", "height", "width", "depth", "temp"]
X = df_new[feature_cols]
y = df_new["faulty"]
# split into train/test sets again
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=15)
# see how imbalanced the label distribution in the training and test sets is
print(f"Fraction of ok items in training set: {1-np.mean(y_train):.3f}")
print(f"Fraction of ok items in test set: {1-np.mean(y_test):.3f}")
# and check the stupid baseline again (this is the same as before since the data contains the same samples)
print("----- Stupid baseline (always predict 'ok'): -----")
print(f"Accuracy on training data: {accuracy_score(y_train, np.zeros_like(y_train)):.3f}")
print(f"Accuracy on test data: {accuracy_score(y_test, np.zeros_like(y_test)):.3f}")
print(f"Balanced accuracy on training data: {balanced_accuracy_score(y_train, np.zeros_like(y_train)):.3f}")
print(f"Balanced accuracy on test data: {balanced_accuracy_score(y_test, np.zeros_like(y_test)):.3f}")

In [ ]:
# train a decision tree again (the parameters here were set as an initial guess
# based on our understanding of the problem as well as the decision tree model)
clf = tree.DecisionTreeClassifier(max_depth=6, min_samples_leaf=50, class_weight="balanced", random_state=1)
clf = clf.fit(X_train, y_train)
print(f"Accuracy on training data: {clf.score(X_train, y_train):.3f}")
print(f"Accuracy on test data: {clf.score(X_test, y_test):.3f}")
print(f"Balanced accuracy on training data: {balanced_accuracy_score(y_train, clf.predict(X_train)):.3f}")
print(f"Balanced accuracy on test data: {balanced_accuracy_score(y_test, clf.predict(X_test)):.3f}")

**Questions:** 
- What do you think of the model now? If anything, is the model over- or underfitting?

In [ ]:
# plot the tree
plt.figure(figsize=(20, 15))
tree.plot_tree(clf, feature_names=feature_cols, filled=True, class_names=np.array(clf.classes_, dtype=str), proportion=True);

As you can see, the tree is quite big and therefore also more tedious to interpret. Additionally, we see that many of the splits right before the leaf nodes are made without any change in the predicted class (e.g., all the nodes remain orange). This happens, because the tree itself only cares about the Gini impurity, which indeed still decreases after these splits. However, since this is not helpful for us, lets prune on the tree by cutting off these unnecessary splits, which can be done by setting the parameter `ccp_alpha`.

In [ ]:
# prune the tree by setting ccp_alpha
clf = tree.DecisionTreeClassifier(max_depth=6, min_samples_leaf=50, class_weight="balanced", ccp_alpha=0.01, random_state=1)
clf = clf.fit(X_train, y_train)
print(f"Accuracy on training data: {clf.score(X_train, y_train):.3f}")
print(f"Accuracy on test data: {clf.score(X_test, y_test):.3f}")
print(f"Balanced accuracy on training data: {balanced_accuracy_score(y_train, clf.predict(X_train)):.3f}")
print(f"Balanced accuracy on test data: {balanced_accuracy_score(y_test, clf.predict(X_test)):.3f}")
# plot the graph
plt.figure(figsize=(15, 10))
tree.plot_tree(clf, feature_names=feature_cols, filled=True, class_names=np.array(clf.classes_, dtype=str), proportion=True);

Notice how the (balanced) accuracy stayed the same after the pruning.

=> Look at this pruned tree and understand which decisions are made (e.g., manually make the same splits on the parallel coordinates plot), i.e., verify that the tree is reaching the same conclusion as we did before.

In [ ]:
# you can also check how each feature influences the prediction
# with a partial dependence plot (works for any model)
plt.figure(figsize=(12, 3))
plot_cols = ['height', 'width', 'depth', 'temp']  # use only continuous variables
display = PartialDependenceDisplay.from_estimator(
    clf, X_train, plot_cols, kind="both", subsample=50, line_kw={"color": '#15317E', "label": None},
    n_cols=4, n_jobs=-1, grid_resolution=20, random_state=13, ax=plt.gca()
)
display.figure_.subplots_adjust(wspace=0.1, hspace=0.5)

### Hyperparameter Tuning

We started out with some initial hyperparameter settings for the decision tree, which already gave us quite good results. However, lets see if we can do even better by systematically testing different hyperparameter combinations, i.e., use a grid search with cross-validation to find an optimal value for `max_depth` and `min_samples_leaf`.

In [ ]:
# to use a grid search, we first need to instantiate our model (including the settings we know we want to use)
clf = tree.DecisionTreeClassifier(class_weight="balanced", ccp_alpha=0.01, random_state=1)
# additionally, we need to define the values we want to try for each parameter 
# (keys in the dict must match the name of the model parameter!)
params = {
    "max_depth": [2, 3, 4, 5, 6, 7, 8],
    "min_samples_leaf": [1, 5, 10, 25, 50, 75, 100, 125]
}
# then pass both the model and the parameter values into the grid search
# normally, the grid search would use the internal .score() function of the model to select the best parameters,
# however, since for a classifier this is the accuracy, we here need to tell the grid search that
# it should select the best model based on the balanced accuracy instead
gs = GridSearchCV(clf, params, scoring='balanced_accuracy')
# the grid search object then can be used like all the other sklearn models
gs.fit(X_train, y_train)
# after it is done, we can check which were the best parameter values
# -> max_depth=5 is what the tree before after pruning had as well
# -> min_samples_leaf=1 does not seem like a good choice 
# (=> always look at the results for all parameter combinations (as we do below), don't just trust the best settings)
print(gs.best_params_)
# and evalute this best model on test set (the grid search already trained the best model
# on the whole dataset for us and we can call .predict() on the grid search object directly)
print(f"Accuracy on training data: {gs.score(X_train, y_train):.3f}")
print(f"Accuracy on test data: {gs.score(X_test, y_test):.3f}")
print(f"Balanced accuracy on training data: {balanced_accuracy_score(y_train, gs.predict(X_train)):.3f}")
print(f"Balanced accuracy on test data: {balanced_accuracy_score(y_test, gs.predict(X_test)):.3f}")

In [ ]:
# overall cross-validation results (lots of stuff...)
gs.cv_results_

In [ ]:
# we're really just interested in the mean test scores for each parameter combination
for i, p in enumerate(gs.cv_results_["params"]):
    print(p, gs.cv_results_["mean_test_score"][i])

In [ ]:
# plot the results as a heatmap to make it easier to see the performance differences
plt.figure()
plt.imshow(gs.cv_results_["mean_test_score"].reshape(len(params["max_depth"]), len(params["min_samples_leaf"])))
plt.colorbar()
plt.xlabel("min_samples_leaf")
plt.ylabel("max_depth")
plt.xticks(range(len(params["min_samples_leaf"])), params["min_samples_leaf"])
plt.yticks(range(len(params["max_depth"])), params["max_depth"])
plt.title("Grid Search Results: Balanced Accuracy");

**Note:** This plot helps us do two things:
1. Verify that the parameter search was exhaustive, i.e., that we've covered a good range of values for each parameter such that it is unlikely that we've missed the best settings in our search.
2. Select the actual parameter values that we want to use for the final model (instead of blindly trusting the values that the grid search had selected for us): notice how with a depth of 5 or greater, all trees with a `min_samples_leaf` setting of 50 or less have the same performance and the grid search simply picked the first model with the best performance. However, as we know a decision tree with a `min_samples_leaf` setting of 1 could in theory memorize individual points, which is not what we want (although this is unlikely with a depth of only 5 and pruning). Therefore, to ensure that we really get robust results, we should instead choose those parameter settings that result in the most regularized model that still produces good results, i.e., in this case a low value for `max_depth` (5) and a high value for `min_samples_leaf` (50).


### Try a different model

Now that we've obtained very good results with a decision tree, lets see if we can do equally well on this dataset with a linear model (i.e., a logistic regression model, since we have a classification problem).

In [ ]:
# try a different classifier: logistic regression
# first, try the model with the default parameter settings
clf = LogisticRegression()
clf = clf.fit(X_train, y_train)
print(f"Accuracy on training data: {clf.score(X_train, y_train):.3f}")
print(f"Accuracy on test data: {clf.score(X_test, y_test):.3f}")
print(f"Balanced accuracy on training data: {balanced_accuracy_score(y_train, clf.predict(X_train)):.3f}")
print(f"Balanced accuracy on test data: {balanced_accuracy_score(y_test, clf.predict(X_test)):.3f}")

In [ ]:
# unbalanced class distributions => set parameter class_weight!! 
# (most sklearn classifiers have this parameter - use it!)
clf = LogisticRegression(class_weight="balanced", random_state=1)
clf = clf.fit(X_train, y_train)
print(f"Accuracy on training data: {clf.score(X_train, y_train):.3f}")
print(f"Accuracy on test data: {clf.score(X_test, y_test):.3f}")
print(f"Balanced accuracy on training data: {balanced_accuracy_score(y_train, clf.predict(X_train)):.3f}")
print(f"Balanced accuracy on test data: {balanced_accuracy_score(y_test, clf.predict(X_test)):.3f}")

The performance is still a lot lower than what we got with a decision tree... Furthermore, you saw in both cases that the model threw a `ConvergenceWarning`. While this usually isn't too tragic in practice (in most cases the results are still quite good), in many cases this warning occurs when the data isn't normally distributed (i.e., violates the model's assumptions) and the results often get better when you transform the data accordingly. Therefore, we now use the `StandardScaler` to ensure each feature has a mean of 0 and a standard deviation of 1.

In [ ]:
# scale the data (but make sure it stays a pandas dataframe)
scaler = StandardScaler().set_output(transform="pandas")
# training data: fit & transform 
# (fit: compute mean and std of each feature; transform: subtract mean from each feature and divide by std)
X_train = scaler.fit_transform(X_train)
# test data: only transform, so the data is comparable!
X_test = scaler.transform(X_test)
# try logreg again -> much better!
clf = LogisticRegression(class_weight="balanced", random_state=1)
clf = clf.fit(X_train, y_train)
print(f"Accuracy on training data: {clf.score(X_train, y_train):.3f}")
print(f"Accuracy on test data: {clf.score(X_test, y_test):.3f}")
print(f"Balanced accuracy on training data: {balanced_accuracy_score(y_train, clf.predict(X_train)):.3f}")
print(f"Balanced accuracy on test data: {balanced_accuracy_score(y_test, clf.predict(X_test)):.3f}")

In [ ]:
# try L1 regularization for feature selection
# (the parameter C determines the strength of the regularization -> smaller values = more regularization)
clf = LogisticRegression(class_weight="balanced", penalty='l1', C=0.1, solver='liblinear', random_state=1)
clf = clf.fit(X_train, y_train)
print(f"Accuracy on training data: {clf.score(X_train, y_train):.3f}")
print(f"Accuracy on test data: {clf.score(X_test, y_test):.3f}")
print(f"Balanced accuracy on training data: {balanced_accuracy_score(y_train, clf.predict(X_train)):.3f}")
print(f"Balanced accuracy on test data: {balanced_accuracy_score(y_test, clf.predict(X_test)):.3f}")

In [ ]:
# the coefficients tell us why an item was classified as faulty:
# higher temperatures lead to faulty items, but we have different offsets for the different products, 
# i.e., product 3 can handle higher temperatures than product 1
# -> features with very small coefficients can be removed
dict(zip(feature_cols, clf.coef_[0]))

In [ ]:
# do a manual feature selection based on the coefficients of the L1 regularized model
feature_cols = ["product_1", "product_17", "temp"]
# construct a new feature matrix and create the train/test split with this new matrix again
X = df_new[feature_cols]
y = df_new["faulty"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=15)
# and don't forget to scale the data again!
scaler = StandardScaler().set_output(transform="pandas")
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
# train the model again with most of the default parameter setting
clf = LogisticRegression(class_weight="balanced", random_state=1)
clf = clf.fit(X_train, y_train)
print(f"Accuracy on training data: {clf.score(X_train, y_train):.3f}")
print(f"Accuracy on test data: {clf.score(X_test, y_test):.3f}")
print(f"Balanced accuracy on training data: {balanced_accuracy_score(y_train, clf.predict(X_train)):.3f}")
print(f"Balanced accuracy on test data: {balanced_accuracy_score(y_test, clf.predict(X_test)):.3f}")
# the performance gets even a tiny bit better, i.e., sometimes less data can be more,
# because additional features can also introduce noise patterns on which a model might overfit

In [ ]:
# by default the logreg model uses L2 regularization with C=1.
# since now we've manually selected the features and know that all of these are important for the task
# we can set C to a higher value to use less regularization
clf = LogisticRegression(class_weight="balanced", penalty='l2', C=1000., random_state=1)
clf = clf.fit(X_train, y_train)
print(f"Accuracy on training data: {clf.score(X_train, y_train):.3f}")
print(f"Accuracy on test data: {clf.score(X_test, y_test):.3f}")
print(f"Balanced accuracy on training data: {balanced_accuracy_score(y_train, clf.predict(X_train)):.3f}")
print(f"Balanced accuracy on test data: {balanced_accuracy_score(y_test, clf.predict(X_test)):.3f}")

While it was a bit more work to set up the logistic regression model appropriately, incl. extra data preprocessing steps, we now even got a balanced accuracy on the test set that is slightly higher than that of the decision tree (0.938 instead of 0.935).